In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../')
from constant import *

In [2]:
class EncoderSentence(nn.Module):
    
    def __init__(self,word_size,word_dim, hidden_size, pretrained_word_embeds=None, output_type = 'sum'):
        super(EncoderSentence, self).__init__()
        
        self.output_type = output_type
        self.word_size = word_size
        self.word_dim = word_dim
        self.hidden_size = hidden_size
        self.pretrained_word_embeds = pretrained_word_embeds
        self.embedding = nn.Embedding(self.word_size,self.word_dim,padding_idx=0)
        self.lstm = nn.LSTM(self.word_dim,self.hidden_size,batch_first = True,bidirectional=True)
        self._init_weights()
        
    def forward(self,x,input_lengths):
        embedded = self.embedding(x)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths,batch_first=True)
        outputs, hidden_cell = self.lstm(packed)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs,batch_first=True)
        #return outputs,hidden_cell
        if self.output_type == 'sum':
            outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        elif self.output_type =='concat':
            outputs = torch.cat((outputs[:, :, :self.hidden_size], outputs[:, : ,self.hidden_size:]),dim=2)
        else:
            raise NotImplementedError 
        return outputs,hidden_cell

    def _init_weights(self):
        if PRE_TRAINED_EMBEDDING or WORD2VEC_EMBEDDING :
            self.embedding.weight.data.copy_(torch.from_numpy(self.pretrained_word_embeds))
            if NON_TRAINABLE:
                self.embedding.weight.requires_grad = False
            else:
                self.embedding.weight.requires_grad = True
        else:
            nn.init.xavier_uniform_(self.embedding.weight.data)

## Test

In [3]:
from load_data_exp import *

In [4]:
for i in train_dataloader:
    break

In [5]:
i[3]

tensor([59, 59, 59, 59, 59, 55, 55, 55, 55, 51, 51, 50, 50, 40, 40, 40, 37, 37,
        30, 30, 30, 30, 29, 29, 28, 28, 28, 22, 22, 18, 18, 14])

In [6]:
enc = EncoderSentence(len(word_mapping)+1,WORD_DIM,128,pretrained_word_embeds,'sum')

In [7]:
e,f = enc(i[1],i[3])

In [8]:
e.size()

torch.Size([32, 59, 128])

In [9]:
e[0].size()

torch.Size([59, 128])

In [10]:
f[0].size()

torch.Size([2, 32, 128])

In [11]:
f[0]

tensor([[[ 0.0338,  0.0332,  0.0210,  ..., -0.0648,  0.0236, -0.0300],
         [ 0.0338,  0.0332,  0.0210,  ..., -0.0648,  0.0236, -0.0300],
         [ 0.0338,  0.0332,  0.0210,  ..., -0.0648,  0.0236, -0.0300],
         ...,
         [ 0.0245,  0.0151,  0.0464,  ..., -0.0081,  0.0179, -0.0393],
         [ 0.0245,  0.0151,  0.0464,  ..., -0.0081,  0.0179, -0.0393],
         [-0.0235, -0.0021,  0.0403,  ..., -0.0370,  0.0243, -0.0374]],

        [[-0.4233,  0.0110,  0.1447,  ...,  0.1305,  0.1711, -0.0455],
         [-0.4233,  0.0110,  0.1447,  ...,  0.1305,  0.1711, -0.0455],
         [-0.4233,  0.0110,  0.1447,  ...,  0.1305,  0.1711, -0.0455],
         ...,
         [-0.2659, -0.0042,  0.0957,  ...,  0.1353,  0.1373, -0.0414],
         [-0.2659, -0.0042,  0.0957,  ...,  0.1353,  0.1373, -0.0414],
         [-0.3920,  0.0092,  0.1386,  ...,  0.1341,  0.1699, -0.0537]]],
       grad_fn=<ViewBackward>)

In [29]:
(f[0].transpose(0,1).transpose(1,2)).contiguous().view(f[0].transpose(0,1).size()[0],-1).size()

torch.Size([32, 256])

In [23]:
(f[0].view(1,-1,256))[0][0][128:]

tensor([ 0.0338,  0.0332,  0.0210, -0.0050,  0.0942, -0.0443,  0.0199, -0.0204,
         0.0860, -0.0108,  0.0372, -0.0430,  0.0004,  0.0066,  0.0158, -0.0360,
        -0.0413, -0.0015, -0.0672, -0.0775, -0.0516,  0.0199, -0.0183, -0.0089,
         0.0288,  0.0352,  0.0141, -0.0541, -0.0444,  0.0337,  0.0214, -0.0277,
         0.0533,  0.0307,  0.0389, -0.0362,  0.0387, -0.0172,  0.0020, -0.0298,
        -0.0013, -0.0397,  0.0080, -0.0435, -0.0059,  0.0558,  0.0454,  0.0708,
        -0.1004,  0.0101,  0.0274, -0.0130, -0.0324, -0.0397,  0.0037, -0.0273,
        -0.0127,  0.0108,  0.0170,  0.0365, -0.0028, -0.0345, -0.0171,  0.0574,
         0.0275,  0.0469, -0.0446,  0.0356, -0.0727,  0.0248, -0.0793,  0.0147,
         0.0122, -0.0518, -0.0254,  0.0066, -0.0292, -0.0437,  0.0338, -0.0062,
         0.0498,  0.0965,  0.0411, -0.0332,  0.0837,  0.0473,  0.0054,  0.0105,
         0.0303, -0.0485,  0.0090, -0.0298,  0.0366, -0.0218, -0.0180,  0.0407,
         0.0018, -0.0700,  0.0774, -0.00

In [24]:
f[0][0:f[0].size(0):2]

tensor([[[ 0.0338,  0.0332,  0.0210,  ..., -0.0648,  0.0236, -0.0300],
         [ 0.0338,  0.0332,  0.0210,  ..., -0.0648,  0.0236, -0.0300],
         [ 0.0338,  0.0332,  0.0210,  ..., -0.0648,  0.0236, -0.0300],
         ...,
         [ 0.0245,  0.0151,  0.0464,  ..., -0.0081,  0.0179, -0.0393],
         [ 0.0245,  0.0151,  0.0464,  ..., -0.0081,  0.0179, -0.0393],
         [-0.0235, -0.0021,  0.0403,  ..., -0.0370,  0.0243, -0.0374]]],
       grad_fn=<SliceBackward>)

In [25]:
f[0][1:f[0].size(0):2]

tensor([[[-0.4233,  0.0110,  0.1447,  ...,  0.1305,  0.1711, -0.0455],
         [-0.4233,  0.0110,  0.1447,  ...,  0.1305,  0.1711, -0.0455],
         [-0.4233,  0.0110,  0.1447,  ...,  0.1305,  0.1711, -0.0455],
         ...,
         [-0.2659, -0.0042,  0.0957,  ...,  0.1353,  0.1373, -0.0414],
         [-0.2659, -0.0042,  0.0957,  ...,  0.1353,  0.1373, -0.0414],
         [-0.3920,  0.0092,  0.1386,  ...,  0.1341,  0.1699, -0.0537]]],
       grad_fn=<SliceBackward>)

In [27]:
h = f[0]
k=torch.cat([h[0:h.size(0):2], h[1:h.size(0):2]], 2)

In [28]:
k.size()

torch.Size([1, 32, 256])